## 1. Import Libraries and Setup

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Text processing
import re
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.layers import (
    Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout, 
    Bidirectional, Embedding, GlobalMaxPooling1D, Concatenate,
    BatchNormalization, Input, Attention, AdditiveAttention
)
from tensorflow.keras.models import Sequential, Model

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)

print(f"TensorFlow version: {tf.__version__}")
print("Libraries imported successfully!")

# PART A: TIME SERIES FORECASTING WITH CNN-LSTM

## 2. Load and Explore Time Series Data

In [ ]:
# Load bike rentals data
df_bikes = pd.read_csv('../../../data/data/FloridaBikeRentals.csv')

print(f"Dataset shape: {df_bikes.shape}")
print(f"\nFirst few rows:")
df_bikes.head()

In [ ]:
# Data information
print("Dataset Information:")
print(df_bikes.info())
print("\n" + "="*80)
print("\nStatistical Summary:")
df_bikes.describe()

In [ ]:
# Parse datetime if needed
if 'Date' in df_bikes.columns:
    df_bikes['Date'] = pd.to_datetime(df_bikes['Date'])
    df_bikes = df_bikes.sort_values('Date')
elif 'datetime' in df_bikes.columns:
    df_bikes['datetime'] = pd.to_datetime(df_bikes['datetime'])
    df_bikes = df_bikes.sort_values('datetime')

# Identify target column (rentals/count)
target_col = None
for col in ['cnt', 'count', 'rentals', 'Count']:
    if col in df_bikes.columns:
        target_col = col
        break

if target_col is None:
    # Use first numeric column as target
    numeric_cols = df_bikes.select_dtypes(include=[np.number]).columns
    target_col = numeric_cols[0]

print(f"Using '{target_col}' as target variable")
print(f"\nTarget statistics:")
print(df_bikes[target_col].describe())

In [ ]:
# Visualize time series
fig, axes = plt.subplots(2, 1, figsize=(18, 10))

# Full time series
axes[0].plot(df_bikes[target_col], linewidth=1)
axes[0].set_xlabel('Time', fontsize=12)
axes[0].set_ylabel('Bike Rentals', fontsize=12)
axes[0].set_title('Bike Rentals Time Series', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Recent data (last 1000 points)
recent_data = df_bikes[target_col].iloc[-1000:]
axes[1].plot(recent_data, linewidth=1.5)
axes[1].set_xlabel('Time (Recent)', fontsize=12)
axes[1].set_ylabel('Bike Rentals', fontsize=12)
axes[1].set_title('Recent Bike Rentals (Last 1000 Points)', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Distribution and patterns
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(df_bikes[target_col], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Bike Rentals', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Bike Rentals', fontsize=14, fontweight='bold')
axes[0].axvline(df_bikes[target_col].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Box plot
axes[1].boxplot(df_bikes[target_col], vert=True)
axes[1].set_ylabel('Bike Rentals', fontsize=12)
axes[1].set_title('Box Plot', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Moving average
window = 24  # 24-hour window
rolling_mean = df_bikes[target_col].rolling(window=window).mean()
axes[2].plot(df_bikes[target_col].iloc[:500], alpha=0.5, label='Original')
axes[2].plot(rolling_mean.iloc[:500], linewidth=2, label=f'{window}h Moving Avg')
axes[2].set_xlabel('Time', fontsize=12)
axes[2].set_ylabel('Bike Rentals', fontsize=12)
axes[2].set_title('Moving Average Pattern', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Prepare Time Series Data for CNN-LSTM

In [ ]:
# Create sequences for time series
def create_sequences(data, seq_length, forecast_horizon=1):
    """
    Create input sequences and target values for time series forecasting.
    
    Args:
        data: Time series data
        seq_length: Length of input sequence
        forecast_horizon: Number of steps to forecast ahead
    """
    X, y = [], []
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length + forecast_horizon - 1])
    return np.array(X), np.array(y)

# Parameters
SEQ_LENGTH = 48  # Use 48 hours to predict next hour
FORECAST_HORIZON = 1  # Predict 1 hour ahead

# Extract target series
time_series = df_bikes[target_col].values

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_scaled = scaler.fit_transform(time_series.reshape(-1, 1)).flatten()

# Create sequences
X, y = create_sequences(time_series_scaled, SEQ_LENGTH, FORECAST_HORIZON)

print(f"Total sequences created: {len(X)}")
print(f"Input shape: {X.shape}")
print(f"Output shape: {y.shape}")

In [ ]:
# Train-test split (temporal)
train_size = int(len(X) * 0.8)
X_train_ts, X_test_ts = X[:train_size], X[train_size:]
y_train_ts, y_test_ts = y[:train_size], y[train_size:]

# Reshape for CNN-LSTM (samples, timesteps, features)
X_train_ts = X_train_ts.reshape((X_train_ts.shape[0], X_train_ts.shape[1], 1))
X_test_ts = X_test_ts.reshape((X_test_ts.shape[0], X_test_ts.shape[1], 1))

print(f"Training set: {X_train_ts.shape}")
print(f"Test set: {X_test_ts.shape}")

## 4. Build CNN-LSTM Model for Time Series

In [ ]:
# CNN-LSTM model
def create_cnn_lstm_model(seq_length, n_features=1):
    model = Sequential([
        # 1D CNN layers for feature extraction
        Conv1D(filters=64, kernel_size=3, activation='relu', 
               input_shape=(seq_length, n_features)),
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        
        # LSTM layers for temporal patterns
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        
        # Dense layers
        Dense(25, activation='relu'),
        Dense(1)
    ])
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Create model
cnn_lstm_model = create_cnn_lstm_model(SEQ_LENGTH)

print("CNN-LSTM Model Architecture:")
cnn_lstm_model.summary()

In [ ]:
# Train model
print("Training CNN-LSTM model...")

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history_cnn_lstm = cnn_lstm_model.fit(
    X_train_ts, y_train_ts,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

print("\nCNN-LSTM training complete!")

## 5. Bidirectional LSTM Model

In [ ]:
# Bidirectional LSTM model
def create_bidirectional_lstm_model(seq_length, n_features=1):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', 
               input_shape=(seq_length, n_features)),
        MaxPooling1D(pool_size=2),
        
        Bidirectional(LSTM(50, return_sequences=True)),
        Dropout(0.3),
        
        Bidirectional(LSTM(50, return_sequences=False)),
        Dropout(0.3),
        
        Dense(25, activation='relu'),
        Dense(1)
    ])
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Create and train
bilstm_model = create_bidirectional_lstm_model(SEQ_LENGTH)

print("Bidirectional LSTM Model:")
print(f"Total parameters: {bilstm_model.count_params():,}")

history_bilstm = bilstm_model.fit(
    X_train_ts, y_train_ts,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

print("Bidirectional LSTM training complete!")

## 6. Evaluate Time Series Models

In [ ]:
# Generate predictions
y_pred_cnn_lstm = cnn_lstm_model.predict(X_test_ts, verbose=0).flatten()
y_pred_bilstm = bilstm_model.predict(X_test_ts, verbose=0).flatten()

# Inverse transform to original scale
y_test_original = scaler.inverse_transform(y_test_ts.reshape(-1, 1)).flatten()
y_pred_cnn_lstm_original = scaler.inverse_transform(y_pred_cnn_lstm.reshape(-1, 1)).flatten()
y_pred_bilstm_original = scaler.inverse_transform(y_pred_bilstm.reshape(-1, 1)).flatten()

# Calculate metrics
def calculate_metrics(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{model_name} Metrics:")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R²: {r2:.4f}")
    print(f"MAPE: {mape:.2f}%")
    
    return {'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}

metrics_cnn_lstm = calculate_metrics(y_test_original, y_pred_cnn_lstm_original, 'CNN-LSTM')
metrics_bilstm = calculate_metrics(y_test_original, y_pred_bilstm_original, 'Bidirectional LSTM')

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 1, figsize=(18, 12))

# CNN-LSTM predictions
plot_range = min(500, len(y_test_original))
axes[0].plot(y_test_original[:plot_range], label='Actual', linewidth=2, alpha=0.7)
axes[0].plot(y_pred_cnn_lstm_original[:plot_range], label='CNN-LSTM Predicted', linewidth=2, alpha=0.7)
axes[0].set_xlabel('Time Steps', fontsize=12)
axes[0].set_ylabel('Bike Rentals', fontsize=12)
axes[0].set_title('CNN-LSTM Model Predictions', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Bidirectional LSTM predictions
axes[1].plot(y_test_original[:plot_range], label='Actual', linewidth=2, alpha=0.7)
axes[1].plot(y_pred_bilstm_original[:plot_range], label='BiLSTM Predicted', linewidth=2, alpha=0.7)
axes[1].set_xlabel('Time Steps', fontsize=12)
axes[1].set_ylabel('Bike Rentals', fontsize=12)
axes[1].set_title('Bidirectional LSTM Model Predictions', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots - Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# CNN-LSTM
axes[0].scatter(y_test_original, y_pred_cnn_lstm_original, alpha=0.5, s=20)
axes[0].plot([y_test_original.min(), y_test_original.max()], 
            [y_test_original.min(), y_test_original.max()], 
            'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual', fontsize=12)
axes[0].set_ylabel('Predicted', fontsize=12)
axes[0].set_title(f'CNN-LSTM (R² = {metrics_cnn_lstm["R2"]:.4f})', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Bidirectional LSTM
axes[1].scatter(y_test_original, y_pred_bilstm_original, alpha=0.5, s=20)
axes[1].plot([y_test_original.min(), y_test_original.max()], 
            [y_test_original.min(), y_test_original.max()], 
            'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual', fontsize=12)
axes[1].set_ylabel('Predicted', fontsize=12)
axes[1].set_title(f'BiLSTM (R² = {metrics_bilstm["R2"]:.4f})', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# PART B: TEXT CLASSIFICATION WITH CNN-LSTM

## 7. Load and Prepare Text Data

In [ ]:
# Try to load real data, otherwise create synthetic dataset
try:
    df_reviews = pd.read_excel('../../../data/data/GrammarandProductReviews.xlsx')
    print("Loaded real review data")
except:
    print("Creating synthetic review dataset for demonstration...")
    
    # Sample reviews
    positive_reviews = [
        "This product is amazing! I love it so much.",
        "Excellent quality and fast shipping. Highly recommend!",
        "Best purchase I've made this year. Very satisfied.",
        "Outstanding product! Exceeded my expectations.",
        "Great value for money. Will buy again."
    ]
    
    negative_reviews = [
        "Terrible product. Waste of money.",
        "Poor quality. Broke after one use.",
        "Not as described. Very disappointed.",
        "Worst purchase ever. Do not buy!",
        "Cheap materials. Not worth the price."
    ]
    
    neutral_reviews = [
        "It's okay. Nothing special.",
        "Average product. Does the job.",
        "Acceptable quality for the price.",
        "Not bad, not great. Just okay.",
        "Decent product. Could be better."
    ]
    
    # Generate more samples
    reviews = []
    ratings = []
    
    for _ in range(200):
        reviews.append(np.random.choice(positive_reviews))
        ratings.append(np.random.choice([4, 5]))
    
    for _ in range(200):
        reviews.append(np.random.choice(negative_reviews))
        ratings.append(np.random.choice([1, 2]))
    
    for _ in range(100):
        reviews.append(np.random.choice(neutral_reviews))
        ratings.append(3)
    
    df_reviews = pd.DataFrame({
        'review': reviews,
        'rating': ratings
    })

print(f"\nReviews dataset shape: {df_reviews.shape}")
df_reviews.head()

In [ ]:
# Identify text and label columns
text_col = None
label_col = None

for col in ['review', 'Review', 'text', 'Text', 'comment', 'Comment']:
    if col in df_reviews.columns:
        text_col = col
        break

for col in ['rating', 'Rating', 'sentiment', 'Sentiment', 'label', 'Label']:
    if col in df_reviews.columns:
        label_col = col
        break

print(f"Using '{text_col}' as text column")
print(f"Using '{label_col}' as label column")

# Convert to binary classification (positive/negative)
if label_col and df_reviews[label_col].dtype in [np.int64, np.float64]:
    median_rating = df_reviews[label_col].median()
    df_reviews['sentiment'] = (df_reviews[label_col] > median_rating).astype(int)
else:
    df_reviews['sentiment'] = df_reviews[label_col]

print(f"\nSentiment distribution:")
print(df_reviews['sentiment'].value_counts())

In [ ]:
# Text preprocessing
def clean_text(text):
    """Clean and preprocess text."""
    if not isinstance(text, str):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

# Clean reviews
df_reviews['clean_review'] = df_reviews[text_col].apply(clean_text)

print("Sample cleaned reviews:")
for i in range(3):
    print(f"\nOriginal: {df_reviews[text_col].iloc[i]}")
    print(f"Cleaned: {df_reviews['clean_review'].iloc[i]}")

## 8. Tokenization and Padding

In [ ]:
# Parameters
MAX_WORDS = 5000
MAX_LEN = 100

# Tokenization
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(df_reviews['clean_review'])

# Convert to sequences
sequences = tokenizer.texts_to_sequences(df_reviews['clean_review'])

# Pad sequences
X_text = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')
y_text = df_reviews['sentiment'].values

print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(f"X_text shape: {X_text.shape}")
print(f"y_text shape: {y_text.shape}")

# Train-test split
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_text, y_text, test_size=0.2, random_state=42, stratify=y_text
)

print(f"\nTraining set: {X_train_text.shape}")
print(f"Test set: {X_test_text.shape}")

## 9. Build Text Classification Models

In [ ]:
# Embedding + CNN + LSTM model
def create_text_cnn_lstm_model(vocab_size, max_len, embedding_dim=100):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_len),
        
        Conv1D(filters=128, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=64, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        
        Dense(32, activation='relu'),
        Dropout(0.3),
        
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Create model
text_model = create_text_cnn_lstm_model(MAX_WORDS, MAX_LEN)

print("Text CNN-LSTM Model:")
text_model.summary()

In [ ]:
# Train model
print("Training text classification model...")

history_text = text_model.fit(
    X_train_text, y_train_text,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\nText model training complete!")

## 10. Model with Attention Mechanism

In [ ]:
# Custom attention layer
class AttentionLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', 
                                shape=(input_shape[-1], 1),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                shape=(input_shape[1], 1),
                                initializer='zeros',
                                trainable=True)
        super(AttentionLayer, self).build(input_shape)
    
    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        e = tf.keras.backend.squeeze(e, axis=-1)
        alpha = tf.keras.backend.softmax(e)
        alpha = tf.keras.backend.expand_dims(alpha, axis=-1)
        context = x * alpha
        context = tf.keras.backend.sum(context, axis=1)
        return context

# Model with attention
def create_text_attention_model(vocab_size, max_len, embedding_dim=100):
    inputs = Input(shape=(max_len,))
    
    x = Embedding(vocab_size, embedding_dim)(inputs)
    x = Conv1D(filters=64, kernel_size=5, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    
    # Attention
    x = AttentionLayer()(x)
    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Create and train
attention_model = create_text_attention_model(MAX_WORDS, MAX_LEN)

print("Text Model with Attention:")
print(f"Total parameters: {attention_model.count_params():,}")

history_attention = attention_model.fit(
    X_train_text, y_train_text,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

print("Attention model training complete!")

## 11. Evaluate Text Classification Models

In [ ]:
# Evaluate models
results_text = text_model.evaluate(X_test_text, y_test_text, verbose=0)
results_attention = attention_model.evaluate(X_test_text, y_test_text, verbose=0)

print("TEXT CLASSIFICATION RESULTS")
print("="*80)

print("\nCNN-LSTM Model:")
print(f"Loss: {results_text[0]:.4f}")
print(f"Accuracy: {results_text[1]:.4f}")
print(f"AUC: {results_text[2]:.4f}")

print("\nCNN-LSTM with Attention:")
print(f"Loss: {results_attention[0]:.4f}")
print(f"Accuracy: {results_attention[1]:.4f}")
print(f"AUC: {results_attention[2]:.4f}")

In [ ]:
# Generate predictions
y_pred_text = (text_model.predict(X_test_text, verbose=0) > 0.5).astype(int).flatten()
y_pred_attention = (attention_model.predict(X_test_text, verbose=0) > 0.5).astype(int).flatten()

# Classification reports
print("\nCNN-LSTM Classification Report:")
print(classification_report(y_test_text, y_pred_text, target_names=['Negative', 'Positive']))

print("\nCNN-LSTM with Attention Classification Report:")
print(classification_report(y_test_text, y_pred_attention, target_names=['Negative', 'Positive']))

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# CNN-LSTM
cm1 = confusion_matrix(y_test_text, y_pred_text)
sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', ax=axes[0],
           xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_title('CNN-LSTM Confusion Matrix', fontsize=14, fontweight='bold')

# With Attention
cm2 = confusion_matrix(y_test_text, y_pred_attention)
sns.heatmap(cm2, annot=True, fmt='d', cmap='Greens', ax=axes[1],
           xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('Actual', fontsize=12)
axes[1].set_title('CNN-LSTM + Attention Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 12. Save Results

In [ ]:
# Save time series predictions
ts_results = pd.DataFrame({
    'Actual': y_test_original,
    'CNN_LSTM_Pred': y_pred_cnn_lstm_original,
    'BiLSTM_Pred': y_pred_bilstm_original
})
ts_path = '../../../data/outputs/time_series_predictions.csv'
ts_results.to_csv(ts_path, index=False)
print(f"Time series predictions saved to: {ts_path}")

# Save text predictions
text_results = pd.DataFrame({
    'Review': df_reviews[text_col].iloc[X_test_text.shape[0]:].values[:len(y_test_text)],
    'Actual': y_test_text,
    'CNN_LSTM_Pred': y_pred_text,
    'Attention_Pred': y_pred_attention
})
text_path = '../../../data/outputs/text_classification_predictions.csv'
text_results.to_csv(text_path, index=False)
print(f"Text predictions saved to: {text_path}")

# Save models
cnn_lstm_model.save('../../../data/outputs/cnn_lstm_timeseries.keras')
text_model.save('../../../data/outputs/cnn_lstm_text.keras')
attention_model.save('../../../data/outputs/cnn_lstm_attention_text.keras')

print("\nAll models and results saved successfully!")

## 13. Summary and Key Findings

In [ ]:
print("="*80)
print("SESSION 11 SUMMARY: CNN-LSTM FOR TIME SERIES AND TEXT")
print("="*80)

print("\nPART A: TIME SERIES FORECASTING")
print("="*80)
print(f"Dataset: Florida Bike Rentals")
print(f"Sequence length: {SEQ_LENGTH} hours")
print(f"Forecast horizon: {FORECAST_HORIZON} hour(s)")

print("\nCNN-LSTM Model:")
for metric, value in metrics_cnn_lstm.items():
    print(f"  {metric}: {value:.4f}" if metric != 'MAPE' else f"  {metric}: {value:.2f}%")

print("\nBidirectional LSTM Model:")
for metric, value in metrics_bilstm.items():
    print(f"  {metric}: {value:.4f}" if metric != 'MAPE' else f"  {metric}: {value:.2f}%")

print("\n" + "="*80)
print("PART B: TEXT CLASSIFICATION")
print("="*80)
print(f"Dataset: Product Reviews")
print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(f"Max sequence length: {MAX_LEN}")

print("\nCNN-LSTM Model:")
print(f"  Accuracy: {results_text[1]:.4f}")
print(f"  AUC: {results_text[2]:.4f}")

print("\nCNN-LSTM with Attention:")
print(f"  Accuracy: {results_attention[1]:.4f}")
print(f"  AUC: {results_attention[2]:.4f}")

print("\n" + "="*80)
print("KEY TECHNIQUES")
print("="*80)
print("1. Hybrid CNN-LSTM Architecture")
print("   - CNN for local pattern extraction")
print("   - LSTM for sequential dependencies")

print("\n2. Bidirectional LSTM")
print("   - Captures past and future context")
print("   - Improved performance on both tasks")

print("\n3. Attention Mechanism")
print("   - Focuses on relevant parts of input")
print("   - Improves interpretability")

print("\n4. Multi-Step Forecasting")
print("   - Sequence-to-sequence prediction")
print("   - Handles temporal patterns")

print("\n" + "="*80)
print("BUSINESS APPLICATIONS")
print("="*80)
print("Time Series:")
print("  - Demand forecasting for resource allocation")
print("  - Predictive maintenance scheduling")
print("  - Inventory optimization")

print("\nText Classification:")
print("  - Customer sentiment analysis")
print("  - Automated review categorization")
print("  - Brand monitoring and reputation management")

print("\n" + "="*80)
print("Session complete! Models ready for deployment.")
print("="*80)